In [ ]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt
import azureml
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.core import ScriptRunConfig
from azureml.core import Model
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


In [ ]:
# Check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

# Select the workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')


# Create an experiment
script_folder = './'
exp = Experiment(workspace=ws, name='keras-ml')

# Load dataset
#dataset = Dataset.get_by_name(ws, 'TrainingSet3K')
dataset = Dataset.get_by_name(ws, 'Sample-data')

# Set the compute node
cluster_name = "standard-ds11"
compute_target = ComputeTarget(workspace=ws, name=cluster_name)

# Specify docker image
keras_env = Environment.from_conda_specification(name = 'keras-2.3.1', file_path = './conda_dependencies.yml')
keras_env.docker.base_image = 'mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest'

# GPU base if GPU compute is used 
#keras_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.0-cudnn7-ubuntu18.04'

docker_config = DockerConfiguration(use_docker=True)

In [ ]:
# Set up and run the experiment
args = ['--input-data', dataset.as_named_input('training_data'), # Reference to dataset
        '--epoch-size', 100] 

src = ScriptRunConfig(source_directory=script_folder,
                      script='keras_model_training.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=keras_env)

from azureml.widgets import RunDetails
run = exp.submit(src)
RunDetails(run).show()
run.wait_for_completion()

In [ ]:
# Register and print model version
model = run.register_model(model_name='model-1', model_path='outputs/model')
print(model.name, 'version', model.version)

In [ ]:
%%writefile score.py
import os
import logging
import json
import numpy
import re
import joblib
from nltk.stem import PorterStemmer
from azureml.core.model import Model
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import model_from_json

# Called when the service is loaded
def init():
    global model
    global tags
    global vectorizer

    model_root = Model.get_model_path('model-1')
    # load json and create model
    json_file = open(os.path.join(model_root, 'model-1.json'), 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(os.path.join(model_root, "model-1.h5"))   
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # load vectorizer
    vectorizer_file = os.path.join(model_root, 'feature_vectorizer.joblib')
    vectorizer = joblib.load(vectorizer_file)

    # load tags
    tags_file = os.path.join(model_root, 'tags.joblib')
    tags = joblib.load(tags_file)

# Called when a request is received
def run(raw_data):
    retured_prediction={}
    logging.info("Request received")
    data = json.loads(raw_data)["data"]
    preprocessed_data = preprocess_data(data)
    result = model.predict(preprocessed_data)
    logging.info("Request processed")
    
    #merge tags with predictions
    combined_result=dict(zip(tags, [str(round(i*100,2)) for i in result[0]]))
    retured_prediction["input_string"]=data
    retured_prediction.update(combined_result)
    return json.dumps(retured_prediction)

# Called when input data is preprocessed
def preprocess_data(received_data):
    stemmer = PorterStemmer()
    returned_text  = re.sub('[\(\*]NA[C]?\s*[\)\(]*C?', r'', received_data, flags = re.M)
    returned_text= " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z0-9]", " ", returned_text).split()]).lower()
    returned_text=stemmer.stem(returned_text)
    pred_text=[]
    pred_text.append(returned_text)
    newX =vectorizer.transform(pred_text).toarray()
    return newX

In [ ]:
# Deploy web service

myenv = Environment.from_conda_specification(name="myenv", file_path="conda_dependencies.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               auth_enabled=False, # this flag generates API keys to secure access
                                               memory_gb=1,
                                               tags={'name': 'New WebService', 'framework': 'Keras'},
                                               description='Keras MLP on text data')

service = Model.deploy(workspace=ws, 
                           name='keras-service', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)

service.wait_for_deployment(True)
print(service.state)



In [ ]:
print(service.get_logs())

In [ ]:
# Test the deployed service
import json

x_new = 'New Text here'


# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)
print(predictions)